<a href="https://colab.research.google.com/github/Rabbicui/DeepLearning/blob/master/%E8%AE%BA%E6%96%87%E4%BB%A3%E7%A0%81%E5%A4%8D%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import division
from __future__ import print_function
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from process import *
from utils import *
from model import *
import uuid

In [2]:
from __future__ import division
from __future__ import print_function
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from utils import *
from model import *
import torch.nn as nn
from sklearn.metrics import f1_score
import uuid

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [39]:
# 参数设置
seed = 42
epochs = 1500
lr = 0.01
weight_decay = 5e-6
layer = 16
hidden = 64
dropout = 0.5
patience = 50
data = 'pubmed'
dev = 0
alpha = 0.1
lamda = 0.5
variant = True


In [ ]:
torch.cuda.is_available()

True

In [40]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

cudaid = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(cudaid)
checkpt_file = uuid.uuid4().hex+'.pt'
print(cudaid,checkpt_file)

cuda ab50f9460dc9470e8a476a569b34420c.pt


In [30]:
def train_step(model,optimizer,features,labels,adj,idx_train):
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(),acc_train.item()

def validate_step(model,features,labels,adj,idx_val):
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test_step(model,features,labels,adj,idx_test):
    model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()
    

In [41]:
def train(datastr,splitstr):
    adj, features, labels, idx_train, idx_val, idx_test, num_features, num_labels = full_load_data(datastr,splitstr)
    features = features.to(device)
    adj = adj.to(device)
    model = GCNII(nfeat=num_features,
                nlayers=layer,
                nhidden=hidden,
                nclass=num_labels,
                dropout=dropout,
                lamda = lamda, 
                alpha=alpha,
                variant=variant).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr,
                            weight_decay=weight_decay)

    bad_counter = 0
    best = 999999999
    for epoch in range(epochs):
        loss_tra,acc_tra = train_step(model,optimizer,features,labels,adj,idx_train)
        loss_val,acc_val = validate_step(model,features,labels,adj,idx_val)
        if(epoch+1)%1 == 0: 
            print('Epoch:{:04d}'.format(epoch+1),
                'train',
                'loss:{:.3f}'.format(loss_tra),
                'acc:{:.2f}'.format(acc_tra*100),
                '| val',
                'loss:{:.3f}'.format(loss_val),
                'acc:{:.2f}'.format(acc_val*100))
        if loss_val < best:
            best = loss_val
            torch.save(model.state_dict(), checkpt_file)
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == patience:
            break
    acc = test_step(model,features,labels,adj,idx_test)[1]

    return acc*100

In [42]:
t_total = time.time()
acc_list = []
# 十折交叉验证
for i in range(10):
    datastr = data
    splitstr = data+'_split_0.6_0.2_'+str(i)+'.npz'
    acc_list.append(train(datastr,splitstr))
    print(i,": {:.2f}".format(acc_list[-1]))
print("Train cost: {:.4f}s".format(time.time() - t_total))
print("Test acc.:{:.2f}".format(np.mean(acc_list)))

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch:0341 train loss:0.287 acc:89.36 | val loss:0.264 acc:90.19
Epoch:0342 train loss:0.290 acc:89.45 | val loss:0.262 acc:90.24
Epoch:0343 train loss:0.288 acc:89.49 | val loss:0.265 acc:90.14
Epoch:0344 train loss:0.289 acc:89.12 | val loss:0.263 acc:90.14
Epoch:0345 train loss:0.286 acc:89.35 | val loss:0.263 acc:90.27
Epoch:0346 train loss:0.288 acc:89.36 | val loss:0.262 acc:90.19
Epoch:0347 train loss:0.276 acc:89.82 | val loss:0.263 acc:90.11
Epoch:0348 train loss:0.289 acc:89.60 | val loss:0.263 acc:90.22
Epoch:0349 train loss:0.289 acc:88.86 | val loss:0.262 acc:90.16
Epoch:0350 train loss:0.281 acc:89.58 | val loss:0.261 acc:90.25
Epoch:0351 train loss:0.284 acc:89.55 | val loss:0.262 acc:90.25
Epoch:0352 train loss:0.278 acc:89.93 | val loss:0.263 acc:90.14
Epoch:0353 train loss:0.277 acc:89.90 | val loss:0.264 acc:90.16
Epoch:0354 train loss:0.280 acc:89.80 | val loss:0.263 acc:90.13
Epoch:0355 train loss:0.281 acc:89.80 | val loss:0.263 acc:90.1